# DarkMoney2020

Isaac Schultz and Adrián Blanco

### Project Description

In recent years, grass-roots giving has become an increasingly popular way of funding political campaigns. The model is self-promoting, as candidates who fundraise independent from political action committee (PAC) spending are considered more transparent. In races like the 2016 presidential election and the 2018 Texas senate election, the immense funds raised by Bernie Sanders (D-Vermont) and Beto O’Rourke (D-Texas) became fundamental aspects of their respective campaigns.

In the run-up to the 2020 election, the question of where presidential candidates get their money from has already become a talking point in media coverage. Our idea is to compile a running list of political action committee (PAC) donations from regularly updated databases, and create a bot that will draw from this data to answer questions for regular people who can’t be bothered to trace any one candidate’s previous donations.

### The Process

* Compile databases to use and create a spreadsheet to pull from.
* Clean data.
* Create a list of addressable queries.
* Probe bot by candidate
* Probe bot by PAC type
* Business
* Labor
* Ideological
* Probe bot by industry
* Probe bot by total funding
* “Sort” functionality
* Code the bot.
* Add a chrome extension via consultation 

### Sources

* OpenSecrets https://www.followthemoney.org/
* Top recipients https://www.opensecrets.org/political-action-committees-pacs/top-recipients
* 2020 Presidential race: https://www.opensecrets.org/2020-presidential-race

* Follow The Money https://www.followthemoney.org/

* FEC Search: https://www.fec.gov/data/search/

* FEC API: https://api.open.fec.gov/developers/ -- Live info, check what's going on in there
  * Example: https://github.com/shmcminn/fec-api-totals-sen/blob/master/api_pull_cands.ipynb

Inspiration
* Election DataBot: https://projects.propublica.org/electionbot/
* FEC Itemizer: https://projects.propublica.org/itemizer/
* nyt-campfinbot: https://github.com/newsdev/nyt-campfinbot

### Week 1 (April 8)

* Pulling the data
* What kind of information can we get from the API?
* What kind of information can we get from the bulk downloads?
* Figure it out which variables, data we are interested in
  * Q of money
  * Dates
  * Name of the candidate
  * Name of the PAC
  * ... TBD

### Week 2  (April 15)

* Structure the data
* Cleaning the data
* First analysis
* April 15, release of new data -> Presidential campaign
* Interface: Bot, Browser extension, graphics and analysis...

### Week 3  (April 22)

* Define the questions we want to ask and answer
  * Look for sth unique that nobody is covering

### Week 4  (April 29)

* Finalize the presentation
  * Notebook and slides

### Week 5  (May 6)

* TBD

* Presentation

### Week 6  (May 13)

* Review
















In [0]:
# Code

We're going to need a bunch of libraries to work with the FEC API.

In [0]:
# Import libraries

import csv
import json

import requests
import pandas as pd
    
    

Then, we set out API key to connect to the API.

In [0]:
api_key = "gfVilw4g272QwuDabu3NQqlcdqUFXqa403VQEHHx"
api_key_2 = "Ejtozq1xEpe3TnAqV536RJUTn5pYUfPdlxbVpMPl"



In [0]:
# https://api.open.fec.gov/developers/
# https://www.fec.gov/help-candidates-and-committees/trainings/

In [0]:
cand_dicts = [{'member': "Beto O'Rourke, D-TX", 'term': '3rd term', 'age': '45', 'name': "Beto O'Rourke", 'party': 'D', 'state': 'TX', 'last_name': "O'Rourke"}]
cand_dicts

cand_dicts

[{'age': '45',
  'last_name': "O'Rourke",
  'member': "Beto O'Rourke, D-TX",
  'name': "Beto O'Rourke",
  'party': 'D',
  'state': 'TX',
  'term': '3rd term'}]

In [0]:
search_url = "https://api.open.fec.gov/v1/candidates/search/"
search_params = {"election_year": "2018",
                 "api_key": api_key,
                 "party": "",
                 "state": "",
                 "office": "S",
                 "name": "",
                 "sort": "name"}

candidate_ids = []

for cand in cand_dicts:
    if cand["party"] == "R":
        search_params["party"] = "REP"
    else:
        search_params["party"] = "DEM"
    search_params["state"] = cand["state"]
    search_params["name"] = cand["last_name"]
    resp = requests.get(search_url, params=search_params)
    d = {}
    d["candidate_name"] = json.loads(resp.text)["results"][0]["name"]
    d["candidate_id"] = json.loads(resp.text)["results"][0]["candidate_id"]
    candidate_ids.append(d)

In [0]:
print(resp)

<Response [200]>


In [0]:
cand_totals = []

filings_params = {"cycle": "2018",
    "api_key": api_key,
    "office": "S",
    "per_page": "60",
    "full_election": "true"
}

for cand in candidate_ids:
    cand_id = cand["candidate_id"]
    filings_url = "https://api.open.fec.gov/v1/candidate/" + cand_id + "/totals/"
    
    resp = requests.get(filings_url, params = filings_params)
    
    total = json.loads(resp.text.replace("null", '""'))
    total["name"] = cand["candidate_name"]
    total["candidate_id"] = cand_id
    
    cand_totals.append(total)

In [0]:
print(cand_totals)

[{'api_version': '1.0', 'pagination': {'count': 1, 'page': 1, 'pages': 1, 'per_page': 60}, 'results': [{'last_beginning_image_number': '201901319145392670', 'other_political_committee_contributions': 83441.3, 'cycle': 2018, 'coverage_end_date': '2018-12-31T00:00:00+00:00', 'other_disbursements': 4616755.58, 'refunded_political_party_committee_contributions': 0.0, 'political_party_committee_contributions': 0.0, 'last_report_type_full': 'YEAR-END', 'offsets_to_fundraising_expenditures': '', 'coverage_start_date': '2017-01-01T00:00:00+00:00', 'last_cash_on_hand_end_period': 286531.29, 'candidate_contribution': 0.0, 'disbursements': 80450312.61, 'operating_expenditures': 74564426.11, 'contributions': 80166657.11, 'loan_repayments_candidate_loans': 0.0, 'last_report_year': 2018, 'individual_contributions': 80083215.81, 'all_other_loans': 0.0, 'fundraising_disbursements': '', 'last_debts_owed_to_committee': 0.0, 'transfers_from_other_authorized_committee': 53666.26, 'refunded_other_political

In [0]:
cands_data = []

for total in cand_totals:
    if len(total["results"]) > 1:
        print("too many results for " + total["name"])
    if len(total["results"]) == 0:
        print("no results for " + total["name"])
        continue
    d = {}
    d["name"] = total["name"]
    for item in total["results"][0]:
        d[item] = total["results"][0][item]  
        
    cands_data.append(d)

In [0]:
print(cands_data)

[{'name': "O'ROURKE, ROBERT (BETO)", 'refunded_individual_contributions': 1264011.92, 'loans': 0.0, 'candidate_contribution': 0.0, 'federal_funds': '', 'candidate_id': 'S8TX00285', 'other_disbursements': 4616755.58, 'offsets_to_operating_expenditures': 89289.43, 'last_beginning_image_number': '201901319145392670', 'transfers_to_other_authorized_committee': 0.0, 'loan_repayments_other_loans': 0.0, 'last_cash_on_hand_end_period': 286531.29, 'disbursements': 80450312.61, 'all_other_loans': 0.0, 'net_contributions': 78897526.19, 'refunded_other_political_committee_contributions': 5117.0, 'offsets_to_legal_accounting': '', 'last_debts_owed_by_committee': 0.0, 'individual_contributions': 80083215.81, 'full_election': True, 'net_operating_expenditures': 74475136.68, 'other_receipts': 28529.56, 'coverage_end_date': '2018-12-31T00:00:00+00:00', 'total_offsets_to_operating_expenditures': '', 'cycle': 2018, 'operating_expenditures': 74564426.11, 'contribution_refunds': 1269128.92, 'individual_ite

In [0]:
for candidate in cands_data:
  print(candidate["name"], candidate["refunded_individual_contributions"])

O'ROURKE, ROBERT (BETO) 1264011.92


In [0]:
search_url = "https://api.open.fec.gov/v1/candidates/search/"
search_params = {"election_year": "2018",
                 "api_key": api_key,
                 "party": "",
                 "state": "",
                 "office": "S",
                 "name": "",
                 "sort": "name"}

In [0]:





url = "https://api.open.fec.gov/v1/committee/C00504530/reports/?page=1&sort=-coverage_end_date&sort_nulls_last=false&per_page=20&type=&sort_null_only=false&sort_hide_null=false&api_key=DEMO_KEY&cycle=2018

SyntaxError: ignored

In [0]:
url = "https://api.open.fec.gov/v1/committee/C00504530/reports/?page=1&sort=-coverage_end_date&sort_nulls_last=false&per_page=20&type=&sort_null_only=false&sort_hide_null=false&api_key=DEMO_KEY&cycle=2018"

resp = requests.get(url)



In [0]:
print(resp)

<Response [200]>


In [0]:
data = resp.json()
print(data)

{'api_version': '1.0', 'pagination': {'page': 1, 'count': 17, 'pages': 1, 'per_page': 20}, 'results': [{'shared_fed_activity_nonfed_ytd': 0.0, 'amendment_indicator_full': 'NEW', 'total_fed_receipts_ytd': 130890564.62, 'transfers_to_affilitated_committees_ytd': 0.0, 'individual_itemized_contributions_ytd': 127789207.99, 'total_operating_expenditures_ytd': 14942036.75, 'loans_made_period': 0.0, 'cash_on_hand_close_ytd': 590831.52, 'independent_expenditures_period': 0.0, 'individual_unitemized_contributions_ytd': 22836.66, 'most_recent_file_number': 1312692.0, 'other_disbursements_ytd': 0.0, 'total_fed_operating_expenditures_period': 329262.92, 'refunded_individual_contributions_period': 0.0, 'cash_on_hand_beginning_period': 701117.02, 'other_political_committee_contributions_ytd': 1745200.0, 'transfers_from_nonfed_levin_ytd': 0.0, 'net_operating_expenditures_ytd': 13608916.78, 'shared_nonfed_operating_expenditures_ytd': 0.0, 'independent_expenditures_ytd': 129212673.38, 'coverage_end_dat

In [0]:
# https://stackoverflow.com/questions/6386308/http-requests-and-json-parsing-in-python/6386366

# Gather data from PACs and Super PACs
# committees

# /committees/

# financial
# /committee/{committee_id}/reports/
# https://api.open.fec.gov/developers/#/financial/get_committee__committee_id__totals_
# curl -X GET "https://api.open.fec.gov/v1/committee/C00504530/reports/?page=1&sort=-coverage_end_date&sort_nulls_last=false&per_page=20&type=&sort_null_only=false&sort_hide_null=false&api_key=DEMO_KEY&cycle=2018" -H "accept: application/json"
# https://api.open.fec.gov/v1/committee/C00504530/reports/?page=1&sort=-coverage_end_date&sort_nulls_last=false&per_page=20&type=&sort_null_only=false&sort_hide_null=false&api_key=DEMO_KEY&cycle=2018



import requests

url = "https://api.open.fec.gov/v1/committee/C00401224/reports/"

params = {
    #"page": 1,
    #'sort': '',
    #'sort_nulls_last': '',
    #'per_page': 1,
    #'type': '',
    #'sort_null_only': '',
    #'sort_hide_null': '',
    'api_key': api_key,
    'cycle': '2018'
}

resp = requests.get(url=url, params=params)
data = resp.json() # Check the JSON Response Content documentation below

print(data)


#for cand in candidate_ids:
#    cand_id = cand["candidate_id"]
#    filings_url = "https://api.open.fec.gov/v1/candidate/" + cand_id + "/totals/"

{'api_version': '1.0', 'results': [{'nonfed_share_allocated_disbursements_period': 0.0, 'cash_on_hand_end_period': 34365223.53, 'refunded_individual_contributions_period': 1499939.03, 'offsets_to_operating_expenditures_ytd': 2067257.85, 'subtotal_summary_ytd': 896936263.74, 'coverage_end_date': '2018-12-31T00:00:00+00:00', 'net_contributions_period': 18809338.22, 'committee_type': 'W', 'total_disbursements_ytd': 862571040.21, 'is_amended': False, 'total_operating_expenditures_ytd': 11169991.96, 'transfers_to_affilitated_committees_ytd': 0.0, 'total_receipts_period': 21409671.42, 'cash_on_hand_close_ytd': 34365223.53, 'fed_candidate_committee_contributions_period': 19245999.5, 'all_loans_received_period': 0.0, 'report_type': 'YE', 'fec_url': 'http://docquery.fec.gov/dcdev/posted/1312107.fec', 'most_recent': True, 'total_fed_election_activity_period': 0.0, 'fed_candidate_committee_contribution_refunds_ytd': 12147641.93, 'loan_repayments_made_period': 0.0, 'refunded_political_party_commit

In [0]:
import pandas as pd
 
df = pd.DataFrame({'key1': [], 'date': [], 'money': []})
for element in data["results"]:
    df = df.append({'date': element["coverage_end_date"], 'money': element["total_receipts_period"]}, ignore_index=True)

In [0]:
time = []
money = []

for element in data["results"]:
  time.append(element["coverage_end_date"].split('T')[0])
  money.append(int(element["total_receipts_period"]))

In [0]:
df["date"] = pd.to_datetime(df['date']) 
df["money"] = pd.to_numeric(df["money"])

In [0]:
time

['2018-12-31',
 '2018-11-26',
 '2018-10-17',
 '2018-09-30',
 '2018-08-31',
 '2018-07-31',
 '2018-06-30',
 '2018-05-31',
 '2018-04-30',
 '2018-03-31',
 '2018-02-28',
 '2018-02-28',
 '2018-01-31',
 '2018-01-31',
 '2017-12-31',
 '2017-12-31',
 '2017-11-30',
 '2017-11-30',
 '2017-10-31',
 '2017-10-31']

In [0]:
import datetime as dt
dates_list = [dt.datetime.strptime(ti, "%Y-%m-%d").date() for ti in time]


In [0]:
dates_list

[datetime.date(2018, 12, 31),
 datetime.date(2018, 11, 26),
 datetime.date(2018, 10, 17),
 datetime.date(2018, 9, 30),
 datetime.date(2018, 8, 31),
 datetime.date(2018, 7, 31),
 datetime.date(2018, 6, 30),
 datetime.date(2018, 5, 31),
 datetime.date(2018, 4, 30),
 datetime.date(2018, 3, 31),
 datetime.date(2018, 2, 28),
 datetime.date(2018, 2, 28),
 datetime.date(2018, 1, 31),
 datetime.date(2018, 1, 31),
 datetime.date(2017, 12, 31),
 datetime.date(2017, 12, 31),
 datetime.date(2017, 11, 30),
 datetime.date(2017, 11, 30),
 datetime.date(2017, 10, 31),
 datetime.date(2017, 10, 31)]

In [0]:
time

['2018-12-31T00:00:00+00:00',
 '2018-11-26T00:00:00+00:00',
 '2018-10-17T00:00:00+00:00',
 '2018-09-30T00:00:00+00:00',
 '2018-08-31T00:00:00+00:00',
 '2018-07-31T00:00:00+00:00',
 '2018-06-30T00:00:00+00:00',
 '2018-05-31T00:00:00+00:00',
 '2018-04-30T00:00:00+00:00',
 '2018-03-31T00:00:00+00:00',
 '2018-02-28T00:00:00+00:00',
 '2018-02-28T00:00:00+00:00',
 '2018-01-31T00:00:00+00:00',
 '2018-01-31T00:00:00+00:00',
 '2017-12-31T00:00:00+00:00',
 '2017-12-31T00:00:00+00:00',
 '2017-11-30T00:00:00+00:00',
 '2017-11-30T00:00:00+00:00',
 '2017-10-31T00:00:00+00:00',
 '2017-10-31T00:00:00+00:00']

Let's see how much money BlueAct collected during 2018

In [0]:
from plotly.plotly import iplot, sign_in
from plotly.graph_objs import Scatter, Figure

# sign into the service (get your own credentials!)
sign_in("cocteautt","8YLww0QuMPVQ46meAMaq")

# create a plot of a single line tracking tweets over time
myplot_parts = [Scatter(x=time,y=money)]

# make a figure from this line plot...
myfigure = Figure(data=myplot_parts)

# ... and plot it (the filename is a convention plotly needs in case you want to use it later)
iplot(myfigure,filename="test1002")

A bar graph is better

In [0]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

# sign into the service (get your own credentials!)
sign_in("cocteautt","8YLww0QuMPVQ46meAMaq")

# create a plot of a single line tracking tweets over time
myplot_parts = [go.Bar(x=time,y=money)]

# make a figure from this line plot...
myfigure = Figure(data=myplot_parts)

# ... and plot it (the filename is a convention plotly needs in case you want to use it later)
iplot(myfigure,filename="test1002")

Candidates data (Dems and Trump):


https://www.fec.gov/data/candidates/president/?election_year=2020&cycle=2020&election_full=true

* Who are the big players in individual contributions

* How much of the money is from outstate. Where does the money come from?

* Any other PACs apart from ActBlue? Super PACs for Trump?

* The percentage of "anonymous" contributions (= ActBlue) as well as for Trump

Committees:

2016 - 2018 data, at least for now

* Super PACs subsectors

* What percentage of contributions are given anonimously v the name attached?

* Which contributors gave the most money

* **IN-KIND contributions**

* Joint contributions: A joint contribution is a contribution that is made by more than one person using a single check or other written instrument. Although each individual has a separate contribution limit, joint contributors may combine their contribution limits by contributing a joint contribution (for example, a check for $5,600 for a candidate’s primary election) as long as both sign the check (or an attached statement).


TASKS: 

Analysis:
* List of PACs and Super PACs
* List of main committeess for each chandidate
    * List of surrounding committees for each candidate
* Compare the fundraising rates to previous campaigns
    * Bernie2020 v Bernie2016
    * Fed election v. Senate election
* List of candidates

Bot:
* List of questions we have and we want to address in the bot


Links:
https://publicintegrity.org/federal-politics/elections/presidential-fundraising-2020-small-dollar-donors-act-blue/

### Questions:
Hi there! I'm Ellie the election bot. I'm here to help you understand 2020 a bit more – there are a ton of candidates, tons of money, and plenty to learn about FEC.

[FEC what?] [I know a bit about this stuff.]

[FEC what?]
The Federal Election Commission! They're the government body in charge of all campaign financing, voter fraud – all things electoral.

[I know a bit about this stuff.]

Gotcha, then we can just get started. What would you like to know about?

[Candidates] [Top spenders] [Surprise Me]

[Candidates]
Sure. Here are some of the candidates running in the 2020 primaries: [CANDIDATES] Who do you want to know more about?

 [BIDEN] [SANDERS] [WARREN] [OTHER]

Ok, you want to learn more about CANDIDATE. [CANDIDATE] Do you want to know more?
 
 [BIDEN]
Joe Biden (D-DE) is a former vice president who currently leads in the Democratic polls. Biden previously sought the Democratic nomination in 1988 and 2008. 

[SANDERS]
Bernie Sanders' (I-VT) campaigns have historically rode on small-money donors. Sanders lost out on the Democratic nomination for president in 2016 to Hillary Clinton, and is caucusing with the Democrats for the 2020 nomination.

[WARREN]
Elizabeth Warren (D-MA) is a former Harvard Law professor whose major policy positions include total student loan forgiveness and breaking up big tech. Warren has sworn off big money fundraisers.

[TELL ME MORE] [SHOW ME OTHER OPTIONS]

[TELL ME MORE - BIDEN]
Biden recently joined the presidential campaign, so there isn't much FEC data on him yet. In April, his campaign said that Biden raised 6.3 million dollars in the first 24 hours, which would be more than any other 2020 democratic candidate.

[TELL ME MORE - SANDERS]
So far, Bernie has raised 18.2 million dollars from individual contributors, the most of any candidate in the 2020 race.


[TELL ME MORE - WARREN]
10.4 million dolalrs of Warren's 2020 war chest has been transferred from leftover funds from her senate race. Legally, only federal election cash can be transferred, giving former congresspeople and senators leverage over governors and mayors.

[OTHER]
[HARRIS] [O'ROURKE] [BUTTIGIEG] [GILLIBRAND] [BOOKER] [KLOBUCHAR] [YANG] [CASTRO] [HICKENLOOPER] [GABBARD] [DELANEY] [INSLEE] [BENNET] [MESSAM] [MOULTON] [RYAN] [SWALWELL] [WILLIAMSON] [TRUMP] [WELD]
  
  [Yes] [No]
 
 [NOT CANDIDATE]
 Hey, stop playing with me. These are the candidates running in the 2020 primaries: [CANDIDATES]. Who do you want to know more about?

[Top spenders]

So far, the candidates who have spent the most of what they've raised are Donald Trump (8.8 million), Elizabeth Warren (5.3 million), and Bernie Sanders (5 million).

[SURPRISE ME]

Ok, check this out. Donors can also give "in-kind" donations, which are non-monetary donations. Elizabeth Warren gave nearly $6000 to her own campaign, in the form of two unique book contributions.
 
TELL ME MORE]
 
Another surprise.

**Why Ellie?**

We named the bot Ellie for a simple reason. The bot is dealing with campaign finance, specifically with the 2020 presidential election in mind. "Ellie" felt like a pretty intuitive abbreviation for election.


**Why a bot?**

Not many people have the time to thoroughly peruse the fillngs on the FEC website. However, many people are interested in campaign finances – where candidate money is coming from, either from small money donors or political action committees, and what that says about the candidates themselves.

A bot is an expedient way for voters to educate themselves on issues of campaign finance, without having to query expansive, jargon-filled databases themselves. Ellie is outfitted to help both novices and experienced individuals when it comes to campaign finance, which can be a nebulous and dense topic.

Ellie also makes the exploration of 2020 candidates' campaign finance an interactive experience. The goal of the bot is to demystify campaign finance and make understanding 2020 money flow easier and (hopefully) a fun thing to do.

**FEC Data**

The Federal Election Commission (FEC) is the regulatory body that administers and polices federal campaign finance law. This makes the FEC the uniquely comprehensive dataset for understanding the money that flows in and out of campaigns, year after year, as well as the citizens and groups that fund different campaigns and causes. Small dollar donations are becoming an increasingly popular means of campaign fundraising, which makes the dataset all the more interesting to query.

* Process

    * To analyze the data, we checked the work done by other reporters.
    
    * Some of the code repos we used as documentation are:
    
        * 
    
    * Finally, we read the FEC API documentation. It was not a very comprehensive data source so we have to take advantage of its sandbox to understand how to deal with FEC data.
    
* Challenges
    * The FEC API documentation is extensive but hard to follow, with over 80 possible endpoints, each one consisting of similar phrasing with technical variations in the information they each pulled. We had to figure out the best endpoint to use for Ellie's purpose, eventually settling on the committee endpoint.
    * Despite FEC quarterly filings being due in mid-April and FEC employees assurance that all data would be processed and accessible via the API within 10 days of the filing deadline, some data is still not fetchable. For this reason, we've had to substitute some queries of 2020 data with queries of previous candidate elections; e.g., Bernie 2016. 



[Narrative Science](https://narrativescience.com/)


### Ellie the Bot


Looking for inspiration, we discovered the following bots that were really useful for our project:

* ProPublica's [Election DataBot](https://projects.propublica.org/electionbot/).
* The New York Times's [Campaign Finance Bot](https://github.com/newsdev/nyt-campfinbot).


These bots was very useful in designing our own.

### Data Sources

* OpenSecrets https://www.opensecrets.org/
* Top recipients https://www.opensecrets.org/political-action-committees-pacs/top-recipients
* 2020 Presidential race: https://www.opensecrets.org/2020-presidential-race

* Follow The Money https://www.followthemoney.org/

* FEC Search: https://www.fec.gov/data/search/

* FEC API: https://api.open.fec.gov/developers/



Also, during this process, we were able to experiment with other languages to build bots.

* First, we learned the basics of Rivescript. RiveScript is a simple scripting language for chatbots with a friendly, easy to learn syntax. It was a useful resource to develop a basic chatbot.

* Then, we moved to Rundexter. Rundexter is a platform that makes easier to run bots in different platforms. For the scope of the class, we designed 


Finally, we made some fireworks art in HTML and CSS to design the website that hosts the project.

TODOS:

* Make Ellie more chatty.
* Update her skills








